In [1]:
!pip install data-refinery 
!pip install pyspark

In [6]:
from datarefinery.tuple.TupleDSL import compose
from datarefinery.TupleOperations import append, substitution, wrap
from datarefinery.FieldOperations import remove_columns, add_column_prefix, column_category
from datarefinery.DateFieldOperations import date_parser, explode_date
from datarefinery.CombineOperations import parallel
import pyspark

In [7]:
weather_categories = ['cold', 'warm', 'hot']

row = {'id': 1, 'fecha': '2017-02-23', 'weather': 'cold'}

# Local

In [9]:
complete_date = compose(
        date_parser(["%Y-%m-%d"]),
        explode_date,
        remove_columns("hour", "minute", "second"),
        add_column_prefix("x")
)

operation = parallel(append(['weather'], column_category(weather_categories)), append(['fecha'], complete_date))

(out, err) = operation(row)

print('Ouput: ', out)

Ouput:  {'cold': 1, 'warm': 0, 'hot': 0, 'x_year': 2017, 'x_month': 2, 'x_day': 23}


# Cluster

In [10]:
sc = pyspark.SparkContext('local[*]')

In [11]:
weather = sc.parallelize([row])
out_rdd = weather.map(operation).map(lambda x: x[0])

print(out_rdd.collect())

[{'cold': 1, 'warm': 0, 'hot': 0, 'x_year': 2017, 'x_month': 2, 'x_day': 23}]
